In [ ]:
# 2023-05-23 - This code was created to quickly backup all of the good `loadedSessPickle.pkl` files from the session folders into the './output' subfolder in the session:
# p: /nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-03_12-3-25/output/loadedSessPickle.pkl already in output folder.
# p: /nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-09_21-17-16/output/loadedSessPickle.pkl already in output folder.
# p: /nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/fet11-04_21-20-3/output/loadedSessPickle.pkl already in output folder.
# p: /nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-03_21-26-8/output/loadedSessPickle.pkl already in output folder.
# p: /nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-09_22-4-5/output/loadedSessPickle.pkl already in output folder.
# ...

In [1]:
%config IPCompleter.use_jedi = False
%load_ext autoreload
%autoreload 3
import os
import sys
from pathlib import Path
import numpy as np
import pandas as pd
from copy import deepcopy
from datetime import datetime, timedelta
from typing import Optional, Dict, List


## Pho's Custom Libraries:
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
from pyphocorehelpers.Filesystem.path_helpers import discover_data_files, generate_copydict, copy_movedict, copy_file, FileList
from pyphocorehelpers.Filesystem.path_helpers import convert_filelist_to_new_parent
from pyphocorehelpers.function_helpers import function_attributes
from pyphocorehelpers.Filesystem.metadata_helpers import FilesystemMetadata, get_files_metadata

# import glob # for finding .whl file after building binary repo
from pyphoplacecellanalysis.General.Batch.runBatch import BatchRun

global_data_root_parent_path = find_first_extant_path([Path(r'/media/MAX/Data'), Path(r'/home/halechr/turbo/Data'), Path(r'W:\Data'), Path(r'/home/halechr/FastData'), Path(r'/Volumes/MoverNew/data')])
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"

completed_pipeline_filename = 'loadedSessPickle.pkl'
completed_global_computations_filename = 'global_computation_results.pkl'

build_module_logger(module_name="Spike3D.pipeline"):
	 Module logger com.PhoHale.Spike3D.pipeline has file logging enabled and will log to EXTERNAL\TESTING\Logging\debug_com.PhoHale.Spike3D.pipeline.log


In [2]:
import ipywidgets as widgets
from ipywidgets import HBox, VBox
from pyphocorehelpers.gui.Jupyter.simple_widgets import build_global_data_root_parent_path_selection_widget
    
all_paths = [Path(r'/home/halechr/turbo/Data'), Path(r'W:\Data'), Path(r'/home/halechr/FastData'), Path(r'/media/MAX/Data'), Path(r'/Volumes/MoverNew/data')]
global_data_root_parent_path = None
def on_user_update_path_selection(new_path: Path):
	global global_data_root_parent_path
	new_global_data_root_parent_path = new_path.resolve()
	global_data_root_parent_path = new_global_data_root_parent_path
	print(f'global_data_root_parent_path changed to {global_data_root_parent_path}')
	assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"
			
global_data_root_parent_path_widget = build_global_data_root_parent_path_selection_widget(all_paths, on_user_update_path_selection)
global_data_root_parent_path_widget

global_data_root_parent_path changed to W:\Data


ToggleButtons(description='Data Root:', layout=Layout(width='auto'), options=(WindowsPath('W:/Data'),), style=…

In [ ]:
global_data_root_parent_path

In [7]:
from neuropy.utils.result_context import IdentifyingContext
from neuropy.core.session.Formats.BaseDataSessionFormats import find_local_session_paths
from pyphocorehelpers.Filesystem.metadata_helpers import FilesystemMetadata, get_file_metadata
from pyphocorehelpers.Filesystem.path_helpers import discover_data_files, generate_copydict, copy_movedict, copy_file, save_copydict_to_text_file, read_copydict_from_text_file, invert_filedict
from pyphoplacecellanalysis.General.Batch.runBatch import get_file_str_if_file_exists
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import check_output_h5_files, copy_files_in_filelist_to_dest
from pyphoplacecellanalysis.General.Batch.runBatch import ConcreteSessionFolder, BackupMethods

# BATCH_DATE_TO_USE = '2023-10-18_Apogee' # used for filenames throught the notebook
# BATCH_DATE_TO_USE = '2023-11-15_Lab' # used for filenames throught the notebook
BATCH_DATE_TO_USE = '2024-04-02_Apogee' # used for filenames throught the notebook


## Old (Pre 2023-09-21):

In [ ]:
# Use glob to find the first generated .whl file in the dist/ directory
found_whl_files = glob.glob('*.whl')
found_whl_files = [a for a in found_whl_files if not a.endswith('current.whl')] # exclude the symlink from the search

In [ ]:
external_computed_ripple_filepath = session.basepath.joinpath('ripple_df.pkl')
## try the '.ripple.npy' ripples:
active_file_suffix = '.ripple.npy'
external_computed_ripple_filepath = fp.with_suffix(active_file_suffix)



In [ ]:
## Moves all previously saved loadedSessPickle.pkl files to the output folder.
correctly_placed_output_files = []
search_path: Path = Path("/home/halechr/turbo/Data/KDIBA/").resolve()
assert search_path.exists()
for p in search_path.rglob(completed_pipeline_filename):
    if p.parent.name != 'output':
        # If not already in the 'output/' subfolder, move it there.
        print(f'p: {p}')
        curr_output_dir = p.parent.joinpath('output')
        curr_output_dir.mkdir(exist_ok=True)
        # print(f'curr_output_dir: {curr_output_dir}')
        new_path = p.replace(curr_output_dir.joinpath(p.name))
        print(f'\t new_path: {new_path}')
        correctly_placed_output_files.append(p)
    else:
        print(f'p: {p} already in output folder.')
        correctly_placed_output_files.append(p)
    
correctly_placed_output_files

In [ ]:
completed_pipeline_filename

In [8]:

## Build Pickle Path:
pkl_path = 'global_batch_result_2023-06-08.pkl'
csv_path = 'global_batch_result_2023-06-08.csv'
h5_path = 'global_batch_result_2023-06-08.h5'

global_batch_result_file_path = Path(global_data_root_parent_path).joinpath(h5_path).resolve() # Use Default

batch_progress_df = BatchRun.load_batch_progress_df_from_h5(global_batch_result_file_path)
batch_progress_df

updated_batch_progress_df = BatchRun.rebuild_basedirs(batch_progress_df, global_data_root_parent_path)
updated_batch_progress_df

updated_good_only_batch_progress_df = updated_batch_progress_df[updated_batch_progress_df['locally_is_ready']].copy()
updated_good_only_batch_progress_df

AttributeError: Can't get attribute 'SessionBatchProgress' on <module 'pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing' from 'C:\\Users\\pho\\repos\\Spike3DWorkEnv\\pyPhoPlaceCellAnalysis\\src\\pyphoplacecellanalysis\\General\\Batch\\NonInteractiveProcessing.py'>

In [ ]:
# Generate code that can be pasted into the current "ReviewOfWork-*.ipynb" notebook to load the good sessions:
print(",\n".join([a_ctxt.get_initialization_code_string() for a_ctxt in updated_good_only_batch_progress_df['context'].to_list()]))


# 2023-09-21 - Mirror Slow Data files to much faster SSD: , Path(r'/home/halechr/FastData')

In [4]:
import os
from pathlib import Path
import shutil # used in `restore_symlink_folder`
from shutil import copytree # used in `make_specific_items_local`
from typing import Optional, Dict, List
from datetime import datetime, timedelta
from pyphocorehelpers.Filesystem.metadata_helpers import FilesystemMetadata, get_file_metadata
from pyphocorehelpers.Filesystem.path_helpers import discover_data_files, generate_copydict, copy_movedict, copy_file
from attrs import define, field, Factory


In [ ]:

a_symlink_path: Path = Path('/home/halechr/FastData/KDIBA/pin01') ## DO NOT CALL .resolve() on the input path, this resolves the symlink!!!
print(f'existing_symlink_folder_path: {a_symlink_path}')
assert a_symlink_path.exists()
symlinked_folder_to_local_folder_containing_symlinks(a_symlink_path, dryrun=False)

In [ ]:
a_symlink_path: Path = Path('/home/halechr/FastData/KDIBA/pin01/one') ## DO NOT CALL .resolve() on the input path, this resolves the symlink!!!
print(f'existing_symlink_folder_path: {a_symlink_path}')
assert a_symlink_path.exists()
symlinked_folder_to_local_folder_containing_symlinks(a_symlink_path, dryrun=False)

In [ ]:
# make_specific_items_local

# existing_symlink_folder_path: Path = Path('/home/halechr/FastData/KDIBA/pin01') ## DO NOT CALL .resolve() on the input path, this resolves the symlink!!!
a_symlink_path: Path = Path('/home/halechr/FastData/KDIBA/pin01/one')
print(f'existing_symlink_folder_path: {a_symlink_path}')
assert a_symlink_path.exists()
desired_local_folder_paths: List[Path] = [Path(v) for v in ['/home/halechr/FastData/KDIBA/vvp01/one/2006-4-18_13-6-1/output/global_computation_results.pkl', '/home/halechr/FastData/KDIBA/vvp01/one/2006-4-18_13-6-1/loadedSessPickle.pkl',
                                                            '/home/halechr/FastData/KDIBA/vvp01/one/2006-4-18_13-6-1/output/global_computation_results.pkl', '/home/halechr/FastData/KDIBA/vvp01/one/2006-4-18_13-6-1/loadedSessPickle.pkl',
                                                            '/home/halechr/FastData/KDIBA/vvp01/one/2006-4-18_13-6-1/output/global_computation_results.pkl', '/home/halechr/FastData/KDIBA/vvp01/one/2006-4-18_13-6-1/loadedSessPickle.pkl',
                                                            ]]
make_specific_items_local(a_symlink_path, desired_local_folder_paths=desired_local_folder_paths, dryrun=True)


In [ ]:
a_symlink_path: Path = Path('/home/halechr/FastData/KDIBA/pin01/one/fet11-01_12-58-54') ## DO NOT CALL .resolve() on the input path, this resolves the symlink!!!
print(f'existing_symlink_path: {a_symlink_path}')
assert a_symlink_path.exists()
symlinked_item_to_full_local_item(a_symlink_path, dryrun=True)


In [ ]:
# existing_symlink_path: Path = Path('/home/halechr/FastData/KDIBA/pin01/one/11-09_22-4-5/11-09_22-4-5IN.5.numclu')
a_symlink_path: Path = Path('/home/halechr/FastData/KDIBA/vvp01/one/2006-4-18_13-6-1/output/global_computation_results.pkl')
print(f'existing_symlink_path: {a_symlink_path}')
assert a_symlink_path.exists()
symlinked_item_to_full_local_item(a_symlink_path, dryrun=False)

In [ ]:
a_symlink_path: Path = Path('/home/halechr/FastData/KDIBA/vvp01/one/2006-4-18_13-6-1/loadedSessPickle.pkl')
print(f'existing_symlink_path: {a_symlink_path}')
assert a_symlink_path.exists()
symlinked_item_to_full_local_item(a_symlink_path, dryrun=False)

In [ ]:
a_symlink_path: Path = Path('/home/halechr/FastData/KDIBA/pin01') ## DO NOT CALL .resolve() on the input path, this resolves the symlink!!!
print(f'existing_symlink_folder_path: {a_symlink_path}')
assert a_symlink_path.exists()
restore_symlink_folder(a_symlink_path, dryrun=False)

In [9]:
def attribute_is_extant_path(instance, attribute, value):
	if not isinstance(value, Path):
		raise TypeError(f"value must be a pathlib.Path but type(value): {type(value)}")
	if not value.exists():
		raise ValueError(f"value must exists on the filesystem!")
	


@define(slots=False)
class DataRepository:
	short_name: str = field()
	host: str = field()
	root_path: Path = field()

	# def __attrs_post_init__(self):
	# 	assert root_path.exists()
		
		
	def try_find_files(self):
		# Find the files and build the movedicts:
		source_data_root = self.root_path.resolve()
		found_session_pickle_files = discover_data_files(source_data_root, glob_pattern='loadedSessPickle.pkl', recursive=True)
		found_global_computation_results_files = discover_data_files(source_data_root, glob_pattern=f'output/{completed_global_computations_filename}', recursive=True)
		return found_session_pickle_files, found_global_computation_results_files

data_repos = {'MAX':DataRepository('MAX', host='LNX00052', root_path=Path(r'/media/MAX/Data')),
'FastData':DataRepository('FastData', host='LNX00052', root_path=Path(r'/home/halechr/FastData')),
'gen_scripts':DataRepository('gen_scripts', host='LNX00052', root_path=Path(r'/home/halechr/FastData/gen_scripts/')),
}


included_session_contexts = [
    # IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15'), # prev completed
    # IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43'), # prev completed
    # IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31'), # prev completed
    # IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19'), # prev completed
    # IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25'),
    # IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40'),
    # IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46'),
    # IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30'),
    # IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-10_12-25-50'),
    # IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-09_16-40-54'),
    # IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-10_12-58-3'),
    # IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44'), # prev completed
    # IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_19-28-0'),
    # IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_12-3-25'),
	  IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_22-4-5'),
    # IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-01_12-58-54'), # prev completed
]

data_repos_good_session_concrete_folders_dict = {}

data_repos
# run_kdiba_gor01_one_2006-6-08_14-26-15

{'MAX': DataRepository(short_name='MAX', host='LNX00052', root_path=WindowsPath('/media/MAX/Data')),
 'FastData': DataRepository(short_name='FastData', host='LNX00052', root_path=WindowsPath('/home/halechr/FastData')),
 'gen_scripts': DataRepository(short_name='gen_scripts', host='LNX00052', root_path=WindowsPath('/home/halechr/FastData/gen_scripts'))}

In [6]:
curr_good_session_concrete_folders = ConcreteSessionFolder.build_concrete_session_folders(data_repos['MAX'].root_path.resolve(), included_session_contexts)
curr_good_session_concrete_folders

AssertionError: global_data_root_parent_path: C:\media\MAX\Data does not exist! Is the right computer's config commented out above?

In [ ]:
included_session_contexts

In [ ]:
data_repos_good_session_concrete_folders_dict['FastData'] = ConcreteSessionFolder.build_concrete_session_folders(data_repos['FastData'].root_path.resolve(), included_session_contexts, allow_create_needed_dirs=True)
data_repos_good_session_concrete_folders_dict['FastData']

In [ ]:
good_session_concrete_folders

In [ ]:
# copy_dict = ConcreteSessionFolder.build_backup_copydict(data_repos_good_session_concrete_folders_dict['FastData'], backup_mode=BackupMethods.RenameInSourceDirectory, rename_backup_suffix=BATCH_DATE_TO_USE, only_include_file_types=['global_pkl'])
copy_dict_FastData = ConcreteSessionFolder.build_backup_copydict(data_repos_good_session_concrete_folders_dict['FastData'], backup_mode=BackupMethods.RenameInSourceDirectory, rename_backup_suffix=BATCH_DATE_TO_USE, only_include_file_types=['local_pkl', 'global_pkl'])
copy_dict_FastData

In [ ]:
moved_files_dict_FastData_files = copy_movedict(copy_dict_FastData)
moved_files_dict_FastData_files

In [ ]:
found_session_pickle_files, found_global_computation_results_files = data_repos['MAX'].try_find_files()
found_session_pickle_files

In [ ]:

source_data_root = Path(r'/media/MAX/Data')
dest_data_root = Path(r'/home/halechr/FastData')
assert source_data_root.exists(), f"source_data_root: {source_data_root} does not exist! Is the right computer's config commented out above?"
assert dest_data_root.exists(), f"dest_data_root: {dest_data_root} does not exist! Is the right computer's config commented out above?"

oldest_modified_date = (datetime.now() - timedelta(days=5))

In [ ]:
# Find the files and build the movedicts:
found_session_pickle_files = discover_data_files(source_data_root, glob_pattern='loadedSessPickle.pkl', recursive=True)
found_global_computation_results_files = discover_data_files(source_data_root, glob_pattern=f'output/{completed_global_computations_filename}', recursive=True)
file_movedict_session_pickle_files = generate_copydict(source_data_root, dest_data_root, found_files=found_session_pickle_files, only_files_newer_than=oldest_modified_date)
file_movedict_global_computation_results_pickle_files = generate_copydict(source_data_root, dest_data_root, found_files=found_global_computation_results_files, only_files_newer_than=oldest_modified_date)

### Actually perform copy operations. This will take a while

In [ ]:
moved_files_dict_session_pickle_files = copy_movedict(file_movedict_session_pickle_files)
moved_files_dict_session_pickle_files

# moved_files_dict = copy_files(filelist_source, filelist_dest)

In [ ]:
moved_files_dict_global_computation_results_pickle_files = copy_movedict(file_movedict_global_computation_results_pickle_files)
moved_files_dict_global_computation_results_pickle_files

# Various file formats:
'global_computation_results_2023-12-07_GL.pkl'
'20231012125859-global_computation_results.pkltmp'
"loadedSessPickle_2023-12-07_GL.pkl"
"backup-20231020190542-loadedSessPickle.pkl"
"loadedSessPickle_2023-10-25_BidirectionalOnly"
"20230912203733-loadedSessPickle.pkl"
"backup-20240125143331-loadedSessPickle.pkl.bak"
"20231129083822-loadedSessPickle.pkl"


In [ ]:
from pyphocorehelpers.Filesystem.path_helpers import parse_unique_file_name
import re
from datetime import datetime

def detect_date_strings(filenames):
    # Regular expression pattern for date (YYYY-MM-DD)
    date_pattern = r'\d{4}-\d{2}-\d{2}'
    # Regular expression pattern for datetime (YYYYMMDDHHMMSS)
    datetime_pattern = r'\d{14}'

    detected_dates = []

    for filename in filenames:
        date_match = re.search(date_pattern, filename)
        datetime_match = re.search(datetime_pattern, filename)

        if date_match:
            date_str = date_match.group()
            # Parse the date string
            try:
                date_value = datetime.strptime(date_str, '%Y-%m-%d')
            except ValueError as e:
                print(f"Error parsing date: {e}")
                date_value = None
            detected_dates.append((date_str, date_value))
        elif datetime_match:
            datetime_str = datetime_match.group()
            # Parse the datetime string
            try:
                datetime_value = datetime.strptime(datetime_str, '%Y%m%d%H%M%S')
            except ValueError as e:
                print(f"Error parsing datetime: {e}")
                datetime_value = None
            detected_dates.append((datetime_str, datetime_value))
        else:
            detected_dates.append((None, None))  # No date/datetime found

    return detected_dates


test_filenames = ['global_computation_results_2023-12-07_GL.pkl',
'20231012125859-global_computation_results.pkltmp',
"loadedSessPickle_2023-12-07_GL.pkl",
"backup-20231020190542-loadedSessPickle.pkl",
"loadedSessPickle_2023-10-25_BidirectionalOnly",
"20230912203733-loadedSessPickle.pkl",
"backup-20240125143331-loadedSessPickle.pkl.bak",
"20231129083822-loadedSessPickle.pkl"]

# [parse_unique_file_name(a_filename) for a_filename in test_filenames]

detected_dates = detect_date_strings(test_filenames)
for filename, (date_str, date_value) in zip(test_filenames, detected_dates):
    print(f'{filename} --> {date_str}, {date_value}')
    

In [29]:

# Hardcoded included_session_contexts:
included_session_contexts = [
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15'), # prev completed
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43'), # prev completed
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31'), # prev completed
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19'), # prev completed
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-10_12-25-50'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-09_16-40-54'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-10_12-58-3'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44'), # prev completed
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_19-28-0'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_12-3-25'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-01_12-58-54'), # prev completed
]

active_global_batch_result_filename=f'global_batch_result_{BATCH_DATE_TO_USE}.pkl'

debug_print = False
known_global_data_root_parent_paths = [Path(r'/home/halechr/cloud/turbo/Data'), Path(r'W:\Data'), Path(r'/nfs/turbo/umms-kdiba/Data'), Path(r'/media/MAX/Data'), Path(r'/Volumes/MoverNew/data')] # , Path(r'/home/halechr/FastData'), Path(r'/nfs/turbo/umms-kdiba/Data'), Path(r'/home/halechr/turbo/Data'), 
global_data_root_parent_path = find_first_extant_path(known_global_data_root_parent_paths)
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"
good_session_concrete_folders = ConcreteSessionFolder.build_concrete_session_folders(global_data_root_parent_path, included_session_contexts)

## Build Slurm Scripts:
session_basedirs_dict: Dict[IdentifyingContext, Path] = {a_session_folder.context:a_session_folder.path for a_session_folder in good_session_concrete_folders}
found_files_dict = {}

for a_ctxt, a_path in session_basedirs_dict.items():
	found_any_kind_session_pickle_files = discover_data_files(a_path, glob_pattern='*loadedSessPickle*.pkl*', recursive=True)
	found_any_global_computation_results_files = discover_data_files(a_path, glob_pattern='output/*global_computation_results*.pkl*', recursive=True)
	found_files_dict[a_ctxt] = (found_any_kind_session_pickle_files, found_any_global_computation_results_files)

found_files_dict


{Context(format_name: 'kdiba'
         animal: 'gor01'
         exper_name: 'one'
         session_name: '2006-6-08_14-26-15'): ([WindowsPath('W:/Data/KDIBA/gor01/one/2006-6-08_14-26-15/20230912203733-loadedSessPickle.pkl'),
   WindowsPath('W:/Data/KDIBA/gor01/one/2006-6-08_14-26-15/20230912203734-loadedSessPickle.pkl'),
   WindowsPath('W:/Data/KDIBA/gor01/one/2006-6-08_14-26-15/20231129081520-loadedSessPickle.pkl'),
   WindowsPath('W:/Data/KDIBA/gor01/one/2006-6-08_14-26-15/20231129084409-loadedSessPickle.pkl'),
   WindowsPath('W:/Data/KDIBA/gor01/one/2006-6-08_14-26-15/loadedSessPickle.pkl'),
   WindowsPath('W:/Data/KDIBA/gor01/one/2006-6-08_14-26-15/loadedSessPickle_2024-03-28_Apogee.pkl')],
  [WindowsPath('W:/Data/KDIBA/gor01/one/2006-6-08_14-26-15/output/global_computation_results.pkl'),
   WindowsPath('W:/Data/KDIBA/gor01/one/2006-6-08_14-26-15/output/global_computation_results_2023-10-06.pkl'),
   WindowsPath('W:/Data/KDIBA/gor01/one/2006-6-08_14-26-15/output/global_computation_

In [30]:
## Just try to find ANY date in the string

flat_found_files_list = []

for a_ctxt, (found_any_kind_session_pickle_files, found_any_global_computation_results_files) in found_files_dict.items():	
    flat_found_files_list.extend(found_any_kind_session_pickle_files)
    flat_found_files_list.extend(found_any_global_computation_results_files)


flat_found_files_list = [str(a_file.as_posix()) for a_file in flat_found_files_list]
flat_found_files_list


['W:/Data/KDIBA/gor01/one/2006-6-08_14-26-15/20230912203733-loadedSessPickle.pkl',
 'W:/Data/KDIBA/gor01/one/2006-6-08_14-26-15/20230912203734-loadedSessPickle.pkl',
 'W:/Data/KDIBA/gor01/one/2006-6-08_14-26-15/20231129081520-loadedSessPickle.pkl',
 'W:/Data/KDIBA/gor01/one/2006-6-08_14-26-15/20231129084409-loadedSessPickle.pkl',
 'W:/Data/KDIBA/gor01/one/2006-6-08_14-26-15/loadedSessPickle.pkl',
 'W:/Data/KDIBA/gor01/one/2006-6-08_14-26-15/loadedSessPickle_2024-03-28_Apogee.pkl',
 'W:/Data/KDIBA/gor01/one/2006-6-08_14-26-15/output/global_computation_results.pkl',
 'W:/Data/KDIBA/gor01/one/2006-6-08_14-26-15/output/global_computation_results_2023-10-06.pkl',
 'W:/Data/KDIBA/gor01/one/2006-6-08_14-26-15/output/global_computation_results_2024-03-28_Apogee.pkl',
 'W:/Data/KDIBA/gor01/one/2006-6-08_14-26-15/output/global_computation_results_with_dt.pkl',
 'W:/Data/KDIBA/gor01/one/2006-6-09_1-22-43/20230524092446-loadedSessPickle.pkl',
 'W:/Data/KDIBA/gor01/one/2006-6-09_1-22-43/20231024174


'W:/Data/KDIBA/gor01/one/2006-6-08_14-26-15/20230912203733-loadedSessPickle.pkl',
'W:/Data/KDIBA/gor01/one/2006-6-08_14-26-15/20230912203734-loadedSessPickle.pkl',
'W:/Data/KDIBA/gor01/one/2006-6-08_14-26-15/20231129081520-loadedSessPickle.pkl',
'W:/Data/KDIBA/gor01/one/2006-6-08_14-26-15/20231129084409-loadedSessPickle.pkl',
'W:/Data/KDIBA/gor01/one/2006-6-08_14-26-15/loadedSessPickle.pkl',
'W:/Data/KDIBA/gor01/one/2006-6-08_14-26-15/loadedSessPickle_2024-03-28_Apogee.pkl',
'W:/Data/KDIBA/gor01/one/2006-6-08_14-26-15/output/global_computation_results.pkl',
'W:/Data/KDIBA/gor01/one/2006-6-08_14-26-15/output/global_computation_results_2023-10-06.pkl',
'W:/Data/KDIBA/gor01/one/2006-6-08_14-26-15/output/global_computation_results_2024-03-28_Apogee.pkl',
'W:/Data/KDIBA/gor01/one/2006-6-08_14-26-15/output/global_computation_results_with_dt.pkl',
'W:/Data/KDIBA/gor01/one/2006-6-09_1-22-43/20230524092446-loadedSessPickle.pkl',
'W:/Data/KDIBA/gor01/one/2006-6-09_1-22-43/20231024174601-loadedSessPickle.pkl',
'W:/Data/KDIBA/gor01/one/2006-6-09_1-22-43/20231031135214-loadedSessPickle.pkl',
'W:/Data/KDIBA/gor01/one/2006-6-09_1-22-43/20231031135546-loadedSessPickle.pkl',
'W:/Data/KDIBA/gor01/one/2006-6-09_1-22-43/20231129083822-loadedSessPickle.pkl',
'W:/Data/KDIBA/gor01/one/2006-6-09_1-22-43/20231129101310-loadedSessPickle.pkl',
'W:/Data/KDIBA/gor01/one/2006-6-09_1-22-43/20231129103113-loadedSessPickle.pkl',
'W:/Data/KDIBA/gor01/one/2006-6-09_1-22-43/20240119173418-loadedSessPickle.pkl',
'W:/Data/KDIBA/gor01/one/2006-6-09_1-22-43/loadedSessPickle.pkl',
'W:/Data/KDIBA/gor01/one/2006-6-09_1-22-43/loadedSessPickle_2024-03-28_Apogee.pkl',
'W:/Data/KDIBA/gor01/one/2006-6-09_1-22-43/loadedSessPickle_GL.pkl',
'W:/Data/KDIBA/gor01/one/2006-6-09_1-22-43/output/global_computation_results.pkl',
'W:/Data/KDIBA/gor01/one/2006-6-09_1-22-43/output/global_computation_results_2023-10-05.pkl',
'W:/Data/KDIBA/gor01/one/2006-6-09_1-22-43/output/global_computation_results_2023-10-06.pkl',
'W:/Data/KDIBA/gor01/one/2006-6-09_1-22-43/output/global_computation_results_2024-03-28_Apogee.pkl',
'W:/Data/KDIBA/gor01/one/2006-6-09_1-22-43/output/global_computation_results_GL.pkl',
'W:/Data/KDIBA/gor01/one/2006-6-12_15-55-31/loadedSessPickle.pkl',
'W:/Data/KDIBA/gor01/one/2006-6-12_15-55-31/loadedSessPickle_2023-10-05.pkl',
'W:/Data/KDIBA/gor01/one/2006-6-12_15-55-31/loadedSessPickle_2023-10-06.pkl',
'W:/Data/KDIBA/gor01/one/2006-6-12_15-55-31/loadedSessPickle_2024-03-28_Apogee.pkl',
'W:/Data/KDIBA/gor01/one/2006-6-12_15-55-31/output/global_computation_results.pkl',
'W:/Data/KDIBA/gor01/one/2006-6-12_15-55-31/output/global_computation_results_2023-10-05.pkl',
'W:/Data/KDIBA/gor01/one/2006-6-12_15-55-31/output/global_computation_results_2023-10-06.pkl',
'W:/Data/KDIBA/gor01/one/2006-6-12_15-55-31/output/global_computation_results_2024-03-28_Apogee.pkl',
'W:/Data/KDIBA/gor01/two/2006-6-07_16-40-19/loadedSessPickle.pkl',
'W:/Data/KDIBA/gor01/two/2006-6-07_16-40-19/loadedSessPickle_2024-03-28_Apogee.pkl',
'W:/Data/KDIBA/gor01/two/2006-6-07_16-40-19/output/global_computation_results.pkl',
'W:/Data/KDIBA/gor01/two/2006-6-07_16-40-19/output/global_computation_results_2023-10-06.pkl',
'W:/Data/KDIBA/gor01/two/2006-6-07_16-40-19/output/global_computation_results_2024-03-28_Apogee.pkl',
'W:/Data/KDIBA/gor01/two/2006-6-08_21-16-25/loadedSessPickle.pkl',
'W:/Data/KDIBA/gor01/two/2006-6-08_21-16-25/loadedSessPickle_2023-10-06.pkl',
'W:/Data/KDIBA/gor01/two/2006-6-08_21-16-25/loadedSessPickle_2024-03-28_Apogee.pkl',
'W:/Data/KDIBA/gor01/two/2006-6-08_21-16-25/output/global_computation_results.pkl',
'W:/Data/KDIBA/gor01/two/2006-6-08_21-16-25/output/global_computation_results_2023-10-05.pkl',
'W:/Data/KDIBA/gor01/two/2006-6-08_21-16-25/output/global_computation_results_2023-10-06.pkl',
'W:/Data/KDIBA/gor01/two/2006-6-08_21-16-25/output/global_computation_results_2024-03-28_Apogee.pkl',
'W:/Data/KDIBA/gor01/two/2006-6-09_22-24-40/20240123021825-loadedSessPickle.pkl',
'W:/Data/KDIBA/gor01/two/2006-6-09_22-24-40/loadedSessPickle.pkl',
'W:/Data/KDIBA/gor01/two/2006-6-09_22-24-40/loadedSessPickle_2024-03-28_Apogee.pkl',
'W:/Data/KDIBA/gor01/two/2006-6-09_22-24-40/output/global_computation_results.pkl',
'W:/Data/KDIBA/gor01/two/2006-6-09_22-24-40/output/global_computation_results_2023-10-05.pkl',
'W:/Data/KDIBA/gor01/two/2006-6-09_22-24-40/output/global_computation_results_2023-10-06.pkl',
'W:/Data/KDIBA/gor01/two/2006-6-09_22-24-40/output/global_computation_results_2024-03-28_Apogee.pkl',
'W:/Data/KDIBA/gor01/two/2006-6-12_16-53-46/loadedSessPickle.pkl',
'W:/Data/KDIBA/gor01/two/2006-6-12_16-53-46/loadedSessPickle_2023-10-06.pkl',
'W:/Data/KDIBA/gor01/two/2006-6-12_16-53-46/loadedSessPickle_2024-03-28_Apogee.pkl',
'W:/Data/KDIBA/gor01/two/2006-6-12_16-53-46/output/global_computation_results.pkl',
'W:/Data/KDIBA/gor01/two/2006-6-12_16-53-46/output/global_computation_results_2023-10-05.pkl',
'W:/Data/KDIBA/gor01/two/2006-6-12_16-53-46/output/global_computation_results_2023-10-06.pkl',
'W:/Data/KDIBA/gor01/two/2006-6-12_16-53-46/output/global_computation_results_2024-03-28_Apogee.pkl',
'W:/Data/KDIBA/vvp01/one/2006-4-09_17-29-30/loadedSessPickle.pkl',
'W:/Data/KDIBA/vvp01/one/2006-4-09_17-29-30/loadedSessPickle_2024-03-28_Apogee.pkl',
'W:/Data/KDIBA/vvp01/one/2006-4-09_17-29-30/output/global_computation_results.pkl',
'W:/Data/KDIBA/vvp01/one/2006-4-09_17-29-30/output/global_computation_results_2023-10-05.pkl',
'W:/Data/KDIBA/vvp01/one/2006-4-09_17-29-30/output/global_computation_results_2023-10-06.pkl',
'W:/Data/KDIBA/vvp01/one/2006-4-09_17-29-30/output/global_computation_results_2024-03-28_Apogee.pkl',
'W:/Data/KDIBA/vvp01/one/2006-4-10_12-25-50/loadedSessPickle.pkl',
'W:/Data/KDIBA/vvp01/one/2006-4-10_12-25-50/loadedSessPickle_2024-03-28_Apogee.pkl',
'W:/Data/KDIBA/vvp01/one/2006-4-10_12-25-50/output/global_computation_results.pkl',
'W:/Data/KDIBA/vvp01/one/2006-4-10_12-25-50/output/global_computation_results_2023-10-05.pkl',
'W:/Data/KDIBA/vvp01/one/2006-4-10_12-25-50/output/global_computation_results_2023-10-06.pkl',
'W:/Data/KDIBA/vvp01/one/2006-4-10_12-25-50/output/global_computation_results_2024-03-28_Apogee.pkl',
'W:/Data/KDIBA/vvp01/two/2006-4-09_16-40-54/loadedSessPickle.pkl',
'W:/Data/KDIBA/vvp01/two/2006-4-09_16-40-54/loadedSessPickle_2023-10-06.pkl',
'W:/Data/KDIBA/vvp01/two/2006-4-09_16-40-54/loadedSessPickle_2024-03-28_Apogee.pkl',
'W:/Data/KDIBA/vvp01/two/2006-4-09_16-40-54/output/global_computation_results.pkl',
'W:/Data/KDIBA/vvp01/two/2006-4-09_16-40-54/output/global_computation_results_2023-10-05.pkl',
'W:/Data/KDIBA/vvp01/two/2006-4-09_16-40-54/output/global_computation_results_2023-10-06.pkl',
'W:/Data/KDIBA/vvp01/two/2006-4-09_16-40-54/output/global_computation_results_2024-03-28_Apogee.pkl',
'W:/Data/KDIBA/vvp01/two/2006-4-10_12-58-3/loadedSessPickle.pkl',
'W:/Data/KDIBA/vvp01/two/2006-4-10_12-58-3/loadedSessPickle_2024-03-28_Apogee.pkl',
'W:/Data/KDIBA/vvp01/two/2006-4-10_12-58-3/output/global_computation_results.pkl',
'W:/Data/KDIBA/vvp01/two/2006-4-10_12-58-3/output/global_computation_results_2023-10-05.pkl',
'W:/Data/KDIBA/vvp01/two/2006-4-10_12-58-3/output/global_computation_results_2023-10-06.pkl',
'W:/Data/KDIBA/vvp01/two/2006-4-10_12-58-3/output/global_computation_results_2024-03-28_Apogee.pkl',
'W:/Data/KDIBA/pin01/one/11-02_17-46-44/20231017151006-loadedSessPickle.pkl',
'W:/Data/KDIBA/pin01/one/11-02_17-46-44/20231113171417-loadedSessPickle.pkl',
'W:/Data/KDIBA/pin01/one/11-02_17-46-44/20231113184042-loadedSessPickle.pkl',
'W:/Data/KDIBA/pin01/one/11-02_17-46-44/loadedSessPickle.pkl',
'W:/Data/KDIBA/pin01/one/11-02_17-46-44/loadedSessPickle_2023-10-05.pkl',
'W:/Data/KDIBA/pin01/one/11-02_17-46-44/loadedSessPickle_2023-10-06.pkl',
'W:/Data/KDIBA/pin01/one/11-02_17-46-44/loadedSessPickle_2024-03-28_Apogee.pkl',
'W:/Data/KDIBA/pin01/one/11-02_17-46-44/output/global_computation_results.pkl',
'W:/Data/KDIBA/pin01/one/11-02_17-46-44/output/global_computation_results_2023-10-05.pkl',
'W:/Data/KDIBA/pin01/one/11-02_17-46-44/output/global_computation_results_2023-10-06.pkl',
'W:/Data/KDIBA/pin01/one/11-02_17-46-44/output/global_computation_results_2024-03-28_Apogee.pkl',
'W:/Data/KDIBA/pin01/one/11-02_19-28-0/20231017152528-loadedSessPickle.pkl',
'W:/Data/KDIBA/pin01/one/11-02_19-28-0/20231017160328-loadedSessPickle.pkl',
'W:/Data/KDIBA/pin01/one/11-02_19-28-0/loadedSessPickle.pkl',
'W:/Data/KDIBA/pin01/one/11-02_19-28-0/loadedSessPickle_2023-10-05.pkl',
'W:/Data/KDIBA/pin01/one/11-02_19-28-0/loadedSessPickle_2023-10-06.pkl',
'W:/Data/KDIBA/pin01/one/11-02_19-28-0/loadedSessPickle_2024-03-28_Apogee.pkl',
'W:/Data/KDIBA/pin01/one/11-02_19-28-0/output/global_computation_results.pkl',
'W:/Data/KDIBA/pin01/one/11-02_19-28-0/output/global_computation_results_2023-10-05.pkl',
'W:/Data/KDIBA/pin01/one/11-02_19-28-0/output/global_computation_results_2023-10-06.pkl',
'W:/Data/KDIBA/pin01/one/11-02_19-28-0/output/global_computation_results_2024-03-28_Apogee.pkl',
'W:/Data/KDIBA/pin01/one/11-03_12-3-25/loadedSessPickle.pkl',
'W:/Data/KDIBA/pin01/one/11-03_12-3-25/loadedSessPickle_2023-10-05.pkl',
'W:/Data/KDIBA/pin01/one/11-03_12-3-25/loadedSessPickle_2023-10-06.pkl',
'W:/Data/KDIBA/pin01/one/11-03_12-3-25/loadedSessPickle_2024-03-28_Apogee.pkl',
'W:/Data/KDIBA/pin01/one/11-03_12-3-25/output/global_computation_results.pkl',
'W:/Data/KDIBA/pin01/one/11-03_12-3-25/output/global_computation_results_2023-10-05.pkl',
'W:/Data/KDIBA/pin01/one/11-03_12-3-25/output/global_computation_results_2023-10-06.pkl',
'W:/Data/KDIBA/pin01/one/11-03_12-3-25/output/global_computation_results_2024-03-28_Apogee.pkl',
'W:/Data/KDIBA/pin01/one/fet11-01_12-58-54/loadedSessPickle.pkl',
'W:/Data/KDIBA/pin01/one/fet11-01_12-58-54/loadedSessPickle_2023-10-05.pkl',
'W:/Data/KDIBA/pin01/one/fet11-01_12-58-54/loadedSessPickle_2023-10-06.pkl',
'W:/Data/KDIBA/pin01/one/fet11-01_12-58-54/loadedSessPickle_2024-03-28_Apogee.pkl',
'W:/Data/KDIBA/pin01/one/fet11-01_12-58-54/output/global_computation_results.pkl',
'W:/Data/KDIBA/pin01/one/fet11-01_12-58-54/output/global_computation_results_2023-10-05.pkl',
'W:/Data/KDIBA/pin01/one/fet11-01_12-58-54/output/global_computation_results_2023-10-06.pkl',
'W:/Data/KDIBA/pin01/one/fet11-01_12-58-54/output/global_computation_results_2024-03-28_Apogee.pkl'

possible_sep_characters = ['_', '-']
'20231017151006-loadedSessPickle.pkl' # hyphen separated
'global_computation_results_2024-03-28_Apogee.pkl' # underscore separated


In [31]:
from pyphocorehelpers.Filesystem.path_helpers import parse_unique_file_name
from pyphocorehelpers.Filesystem.path_helpers import try_parse_chain
    
# basename: str = _test_h5_filename.stem
# final_parsed_output_dict = try_parse_chain(basename=basename)

# '20231017151006-loadedSessPickle.pkl' # hyphen separated

# [parse_unique_file_name(a_file) for a_file in flat_found_files_list]


parsed_file_results = {}
failed_parse_files = []

for a_file in flat_found_files_list:
    a_parse_result_attempt = try_parse_chain(basename=Path(a_file).resolve().stem)
    if a_parse_result_attempt is not None:
        parsed_file_results[a_file] = a_parse_result_attempt
    else:
        failed_parse_files.append(a_file)

# [try_parse_chain(basename=Path(a_file).resolve().stem) for a_file in flat_found_files_list]

# parsed_file_results
failed_parse_files


['W:/Data/KDIBA/gor01/one/2006-6-08_14-26-15/loadedSessPickle.pkl',
 'W:/Data/KDIBA/gor01/one/2006-6-08_14-26-15/loadedSessPickle_2024-03-28_Apogee.pkl',
 'W:/Data/KDIBA/gor01/one/2006-6-08_14-26-15/output/global_computation_results.pkl',
 'W:/Data/KDIBA/gor01/one/2006-6-08_14-26-15/output/global_computation_results_2023-10-06.pkl',
 'W:/Data/KDIBA/gor01/one/2006-6-08_14-26-15/output/global_computation_results_2024-03-28_Apogee.pkl',
 'W:/Data/KDIBA/gor01/one/2006-6-08_14-26-15/output/global_computation_results_with_dt.pkl',
 'W:/Data/KDIBA/gor01/one/2006-6-09_1-22-43/loadedSessPickle.pkl',
 'W:/Data/KDIBA/gor01/one/2006-6-09_1-22-43/loadedSessPickle_2024-03-28_Apogee.pkl',
 'W:/Data/KDIBA/gor01/one/2006-6-09_1-22-43/loadedSessPickle_GL.pkl',
 'W:/Data/KDIBA/gor01/one/2006-6-09_1-22-43/output/global_computation_results.pkl',
 'W:/Data/KDIBA/gor01/one/2006-6-09_1-22-43/output/global_computation_results_2023-10-05.pkl',
 'W:/Data/KDIBA/gor01/one/2006-6-09_1-22-43/output/global_computatio

In [49]:

loadedSessPickle_test_strings = [
'loadedSessPickle.pkl',
'loadedSessPickle_2023-10-06.pkl',
'loadedSessPickle_2024-03-28_Apogee.pkl',
]

global_computation_results_test_strings = [
'global_computation_results.pkl',
'global_computation_results_2023-10-06.pkl',
'global_computation_results_2024-03-28_Apogee.pkl',
]



# [try_datetime_detect_by_split(a_file) for a_file in loadedSessPickle_test_strings]
# [try_datetime_detect_by_split(a_file) for a_file in global_computation_results_test_strings]

[try_detect_full_file_export_filename(a_file) for a_file in loadedSessPickle_test_strings]


[{'export_file_type': 'loadedSessPickle'},
 {'export_file_type': 'loadedSessPickle',
  'detected_datetime': datetime.datetime(2023, 10, 6, 0, 0)},
 {'export_file_type': 'loadedSessPickle',
  'detected_datetime': datetime.datetime(2024, 3, 28, 0, 0),
  'variant_suffix': 'Apogee'}]

In [50]:
[try_detect_full_file_export_filename(a_file) for a_file in global_computation_results_test_strings]


[{'export_file_type': 'global_computation_results'},
 {'export_file_type': 'global_computation_results',
  'detected_datetime': datetime.datetime(2023, 10, 6, 0, 0)},
 {'export_file_type': 'global_computation_results',
  'detected_datetime': datetime.datetime(2024, 3, 28, 0, 0),
  'variant_suffix': 'Apogee'}]

In [11]:
def render_tree(found_files_dict):
    tree_str = ""
    for session, (session_files, global_files) in found_files_dict.items():
        tree_str += f"{session}\n"
        if session_files:
            tree_str += "  - Session Pickle Files:\n"
            for file in session_files:
                tree_str += f"    * {file}\n"
        if global_files:
            tree_str += "  - Global Computation Results Files:\n"
            for file in global_files:
                tree_str += f"    * {file}\n"
    return tree_str

# Example of usage
tree_representation = render_tree(found_files_dict)
print(tree_representation)

kdiba_gor01_one_2006-6-08_14-26-15
  - Session Pickle Files:
    * W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\20230912203733-loadedSessPickle.pkl
    * W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\20230912203734-loadedSessPickle.pkl
    * W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\20231129081520-loadedSessPickle.pkl
    * W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\20231129084409-loadedSessPickle.pkl
    * W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\loadedSessPickle.pkl
    * W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\loadedSessPickle_2024-03-28_Apogee.pkl
  - Global Computation Results Files:
    * W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\output\global_computation_results.pkl
    * W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\output\global_computation_results_2023-10-06.pkl
    * W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\output\global_computation_results_2024-03-28_Apogee.pkl
    * W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\output\global_computation_results_with_dt.pkl
kdiba_gor01_one_20

In [14]:
def create_tree_select_nodes(found_files_dict):
    """ 
    {
        "label": "Folder B",
        "value": "folder_b",
        "children": [
            {"label": "Sub-folder A", "value": "sub_a"},
            {"label": "Sub-folder B", "value": "sub_b"},
            {"label": "Sub-folder C", "value": "sub_c"},
        ],
    },

    """
    nodes = []
    for i, (session, (session_files, global_files)) in enumerate(found_files_dict.items()):
        a_session_node = {"label": str(session), "value": str(session),
         'children':
            [
                {"label": 'Session Pickle Files', "value": 'Session Pickle Files',
                'children':
                    [{"label": str(file), "value": str(file)} for file in session_files],
                },
                {"label": 'Global Computation Results Files', "value": 'Global Computation Results Files',
                'children':
                    [{"label": str(file), "value": str(file)} for file in global_files],
                },
            ]
        }
        nodes.append(a_session_node)

    return nodes

# Example usage
nodes = create_tree_select_nodes(found_files_dict)
nodes

[{'label': 'kdiba_gor01_one_2006-6-08_14-26-15',
  'value': 'kdiba_gor01_one_2006-6-08_14-26-15',
  'children': [{'label': 'Session Pickle Files',
    'value': 'Session Pickle Files',
    'children': [{'label': 'W:\\Data\\KDIBA\\gor01\\one\\2006-6-08_14-26-15\\20230912203733-loadedSessPickle.pkl',
      'value': 'W:\\Data\\KDIBA\\gor01\\one\\2006-6-08_14-26-15\\20230912203733-loadedSessPickle.pkl'},
     {'label': 'W:\\Data\\KDIBA\\gor01\\one\\2006-6-08_14-26-15\\20230912203734-loadedSessPickle.pkl',
      'value': 'W:\\Data\\KDIBA\\gor01\\one\\2006-6-08_14-26-15\\20230912203734-loadedSessPickle.pkl'},
     {'label': 'W:\\Data\\KDIBA\\gor01\\one\\2006-6-08_14-26-15\\20231129081520-loadedSessPickle.pkl',
      'value': 'W:\\Data\\KDIBA\\gor01\\one\\2006-6-08_14-26-15\\20231129081520-loadedSessPickle.pkl'},
     {'label': 'W:\\Data\\KDIBA\\gor01\\one\\2006-6-08_14-26-15\\20231129084409-loadedSessPickle.pkl',
      'value': 'W:\\Data\\KDIBA\\gor01\\one\\2006-6-08_14-26-15\\20231129084409-

In [22]:
import ipywidgets as widgets

def create_tree_view(found_files_dict):
    """ creates a working tree view that displays the sessions and their found files, with the found files as checkable items """
    checkbox_layout = widgets.Layout(width='auto', flex='1 1 auto')
    sessions_accordion = widgets.Accordion(children=[widgets.VBox() for _ in found_files_dict])
    for i, (session, (session_files, global_files)) in enumerate(found_files_dict.items()):
        session_accordion = sessions_accordion.children[i]

        # Convert paths to strings and create session pickle files checkboxes
        session_files_checkboxes = [widgets.Checkbox(description=str(file), value=False, layout=checkbox_layout) for file in session_files]
        session_files_vbox = widgets.VBox(children=session_files_checkboxes)

        # Create session pickle files accordion
        session_files_accordion = widgets.Accordion(children=[session_files_vbox])
        session_files_accordion.set_title(0, 'Session Pickle Files')

        # Convert paths to strings and create global computation results files checkboxes
        global_files_checkboxes = [widgets.Checkbox(description=str(file), value=False, layout=checkbox_layout) for file in global_files]
        global_files_vbox = widgets.VBox(children=global_files_checkboxes)

        # Create global computation results files accordion
        global_files_accordion = widgets.Accordion(children=[global_files_vbox])
        global_files_accordion.set_title(0, 'Global Computation Results Files')

        # Set session accordion children
        session_accordion.children = [session_files_accordion, global_files_accordion]
        
        # Set main accordion title
        sessions_accordion.set_title(i, str(session))
        

    return sessions_accordion

def get_selected_items(sessions_accordion):
    selected_items = {}

    for i in range(len(sessions_accordion.children)):
        session_title = sessions_accordion.get_title(i)
        session_vbox = sessions_accordion.children[i]
        session_files_accordion = session_vbox.children[0]
        global_files_accordion = session_vbox.children[1]

        # Get selected session files
        selected_session_files = [cb.description for cb in session_files_accordion.children[0].children if cb.value]
        # Get selected global files
        selected_global_files = [cb.description for cb in global_files_accordion.children[0].children if cb.value]

        selected_items[session_title] = {
            'session_files': selected_session_files,
            'global_files': selected_global_files
        }

    return selected_items



# Example usage
tree_view = create_tree_view(found_files_dict)
# tree_view

button = widgets.Button(description="Get Selected Items")
# Button on_click event handler
def on_button_clicked(b):
    selected_items = get_selected_items(tree_view)
    print(selected_items)  # You can replace this with any action using the selected items

button.on_click(on_button_clicked)

display(widgets.VBox([tree_view, button]))  # Show the button in your notebook


In [ ]:

# search_parent_path = Path(r'W:\\Data\\Kdiba')
# search_parent_path = Path(r'/home/halechr/turbo/Data/KDIBA').resolve() # Greatlakes
search_parent_path = Path(r'/home/halechr/cloud/turbo/Data/KDIBA').resolve() # Lab

# found_autoversioned_session_pickle_files = discover_data_files(search_parent_path, glob_pattern='*-loadedSessPickle.pkl', recursive=True)
# found_global_computation_results_files = discover_data_files(search_parent_path, glob_pattern='output/*.pkl', recursive=True)


found_any_kind_session_pickle_files = discover_data_files(search_parent_path, glob_pattern='*loadedSessPickle*.pkl', recursive=True)
found_any_global_computation_results_files = discover_data_files(search_parent_path, glob_pattern='output/*global_computation_results*.pkl', recursive=True)

# found_session_pickle_files = discover_data_files(source_data_root, glob_pattern='loadedSessPickle.pkl', recursive=True)
# found_global_computation_results_files = discover_data_files(source_data_root, glob_pattern=f'output/{completed_global_computations_filename}', recursive=True)

found_files = found_any_global_computation_results_files + found_any_kind_session_pickle_files
found_files

# 2023-11-20 - Discovery of folder modification times from matching files

In [ ]:
Write a python function that


In [ ]:

def determine_directory_modification_time(target_